In [1]:
from pyspark import SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, IntegerType, DateType, StringType, StructField, BooleanType, TimestampType, FloatType
from pyspark.sql import functions as F

import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D
from tensorflow.keras import backend as K

from sparknlp.annotator import * 
from sparknlp.common import * 
from sparknlp.base import * 

In [2]:
# Define schema 
sarcasm_schema = StructType([StructField("article_link", StringType(), True), StructField("headline", StringType(), True), StructField("is_sarcastic", IntegerType(), True)])


In [3]:
# Import labeled sarcasm dataset from uploaded json file
sarcasm_df = spark.read.format("json").load("/FileStore/tables/Sarcasm_Headlines_Dataset.json", schema=sarcasm_schema)

In [4]:
sarcasm_df.show()

+--------------------+--------------------+------------+
 article_link| headline|is_sarcastic|
+--------------------+--------------------+------------+
https://www.huffi...|former versace st...| 0|
https://www.huffi...|the 'roseanne' re...| 0|
https://local.the...|mom starting to f...| 1|
https://politics....|boehner just want...| 1|
https://www.huffi...|j.k. rowling wish...| 0|
https://www.huffi...|advancing the wor...| 0|
https://www.huffi...|the fascinating c...| 0|
https://www.huffi...|this ceo will sen...| 0|
https://politics....|top snake handler...| 1|
https://www.huffi...|friday's morning ...| 0|
https://www.huffi...|airline passenger...| 0|
https://www.huffi...|facebook reported...| 0|
https://www.huffi...|north korea prais...| 0|
https://www.huffi...|actually, cnn's j...| 0|
https://www.huffi...|barcelona holds h...| 0|
https://entertain...|nuclear bomb deto...| 1|
https://www.theon...|cosby lawyer asks...| 1|
https://www.theon...|stock analysts co...| 1|
https://www.huffi...|bloomberg's progr...| 0|
https://www.huffi...|craig hicks indicted| 0|
+--------------------+--------------------+------------+
only showing top 20 rows

In [5]:
# Create doc, sentence, and token pipeline stages using headline as input
docs = DocumentAssembler().setInputCol("headline").setOutputCol("doc")
sentences = SentenceDetector().setInputCols(["doc"]).setOutputCol("sentence")
tokens = Tokenizer().setInputCols(["sentence"]).setOutputCol("token")

In [6]:
# Create the pipeline
sar_pipeline = Pipeline(stages=[docs, sentences, tokens])

In [7]:
# Fit the dataframe to the pipeline, transform and persist
sar_model = sar_pipeline.fit(sarcasm_df)
processed = sar_model.transform(sarcasm_df).persist()
processed.count()

Out[58]: 26709

In [8]:
# Build test and trainging set split
train, test = processed.randomSplit(weights=[0.75, 0.25], seed=25)
print(train.count())
print(test.count())

19997
6712

In [9]:
# Use pretrained word embeddings model and transform against both train and test features
# May need to dlownload glove_100d file locally on first run
glove = WordEmbeddingsModel.pretrained()
train_features = glove.transform(train)
test_features = glove.transform(test)

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ | ][OK!]

In [10]:
# Convert train/test sets to pandas df to convert into numpy array for model training
x_train_df = train_features.select(['embeddings']).toPandas()
y_train_df = train_features.select(['is_sarcastic']).toPandas()
x_test_df = test_features.select(['embeddings']).toPandas()
y_test_df = test_features.select(['is_sarcastic']).toPandas()

In [11]:
# Create UDF to append embeddings values to result array and return as numpy array
def get_features(row):
  result = []
  for col in row:
    result.append(col['embeddings'])
  return np.array(result)

In [12]:
# Call get_feature UDF
x_train_df['features'] = x_train_df['embeddings'].apply(get_features)
x_test_df['features'] = x_test_df['embeddings'].apply(get_features)

In [13]:
# Extract numpy arrays for features and is_sarcastic values for use as inputs into model
x_train = x_train_df['features'].values
x_test = x_test_df['features'].values
y_train = y_train_df['is_sarcastic'].values
y_test = y_test_df['is_sarcastic'].values

In [14]:
# set parameters for our model:
maxlen = 150 #max 50 words per article
batch_size = 48 #size of the batch 
filters = 100 #dimension of filters for the convolutional layer
kernel_size = 3 #size of the kernel used in the convolutional layer
hidden_dims = 250 #dimension of the hidden layer
epochs = 10 #number of training epochs

In [15]:
# Pad train and test features to same size (max length) and cast resulting arrays to float32 (used for keras support)
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

In [16]:
# Create base model
model = Sequential()

In [17]:
# Add a Conv1D as first layer and
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# Add hidden layers
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))


In [18]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy','mae', 'mse'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Epoch 1/10
 1/417 [..............................] - ETA: 0s - loss: 0.6805 - accuracy: 0.5833 - mae: 0.4802 - mse: 0.2437 3/417 [..............................] - ETA: 9s - loss: 0.6998 - accuracy: 0.5278 - mae: 0.4937 - mse: 0.2531 5/417 [..............................] - ETA: 11s - loss: 0.6965 - accuracy: 0.5250 - mae: 0.4949 - mse: 0.2515 7/417 [..............................] - ETA: 12s - loss: 0.6975 - accuracy: 0.5179 - mae: 0.4967 - mse: 0.2520 9/417 [..............................] - ETA: 12s - loss: 0.6930 - accuracy: 0.5208 - mae: 0.4936 - mse: 0.2499 11/417 [..............................] - ETA: 12s - loss: 0.6889 - accuracy: 0.5322 - mae: 0.4890 - mse: 0.2480 13/417 [..............................] - ETA: 13s - loss: 0.6851 - accuracy: 0.5449 - mae: 0.4844 - mse: 0.2461 15/417 [>.............................] - ETA: 13s - loss: 0.6892 - accuracy: 0.5375 - mae: 0.4852 - mse: 0.2481 17/417 [>.............................] - ETA: 13s - loss: 0.6898 - accuracy: 0.5380 - mae: 0.4862 - mse: 0.2484 19/417 [>.............................] - ETA: 13s - loss: 0.6910 - accuracy: 0.5263 - mae: 0.4878 - mse: 0.2489 21/417 [>.............................] - ETA: 13s - loss: 0.6905 - accuracy: 0.5327 - mae: 0.4884 - mse: 0.2487 23/417 [>.............................] - ETA: 13s - loss: 0.6887 - accuracy: 0.5399 - mae: 0.4879 - mse: 0.2478 25/417 [>.............................] - ETA: 13s - loss: 0.6881 - accuracy: 0.5367 - mae: 0.4874 - mse: 0.2475 27/417 [>.............................] - ETA: 13s - loss: 0.6874 - accuracy: 0.5386 - mae: 0.4868 - mse: 0.2472 29/417 [=>............................] - ETA: 13s - loss: 0.6841 - accuracy: 0.5460 - mae: 0.4851 - mse: 0.2456 31/417 [=>............................] - ETA: 13s - loss: 0.6833 - accuracy: 0.5457 - mae: 0.4847 - mse: 0.2452 33/417 [=>............................] - ETA: 12s - loss: 0.6829 - accuracy: 0.5461 - mae: 0.4845 - mse: 0.2450 35/417 [=>............................] - ETA: 12s - loss: 0.6807 - accuracy: 0.5536 - mae: 0.4837 - mse: 0.2439

In [19]:
%sh
rm -rf /tmp/sarcasm
mkdir /tmp/sarcasm

In [20]:

path = '/FileStore/sarcasm.h5'
model.save(path) 



In [21]:


dbutils.fs.cp("file:/tmp/sarcasm.h5", "dbfs:/FileStore/sarcasm.h5")
display(dbutils.fs.ls("dbfs:/FileStore/sarcasm.h5"))

path,name,size
dbfs:/FileStore/sarcasm.h5,sarcasm.h5,703448


In [22]:
https://community.cloud.databricks.com/files/tmp/sarcasm.h5?o=6852060676645499